# 02 - Limpieza y Feature Engineering a nivel cliente

En este notebook:
1. Se limpian las transacciones (filtros sobre `CustomerID`, `Quantity`, `Price`).
2. Se generan variables agregadas por cliente (RFM y otras).
3. Se guarda el dataset `customer_features.csv` para usar en el clustering.


In [1]:
import pandas as pd
import numpy as np

# Cargar data cruda
df = pd.read_csv("../data/raw/online_retail_II.csv")
df.columns = ["Invoice","StockCode","Description","Quantity",
              "InvoiceDate","Price","CustomerID","Country"]

# Convertir fecha
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# Eliminar filas sin fecha válida
df = df.dropna(subset=["InvoiceDate"])

# Eliminar filas sin CustomerID
df = df.dropna(subset=["CustomerID"])

# CustomerID a entero
df["CustomerID"] = df["CustomerID"].astype(int)

# Filtrar transacciones positivas
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)]

# Calcular importe de línea
df["LineTotal"] = df["Quantity"] * df["Price"]

df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,LineTotal
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0


In [2]:
df.to_csv("../data/processed/transactions_clean.csv", index=False)
len(df)


805549

Las transacciones ya están limpias y guardadas en:

`data/processed/transactions_clean.csv`

A continuación se construyen las variables a nivel cliente.


In [3]:
# Agrupación por cliente
grp = df.groupby("CustomerID")

reference_date = df["InvoiceDate"].max()

customer = pd.DataFrame()

customer["frequency"]   = grp["Invoice"].nunique()
customer["monetary"]    = grp["LineTotal"].sum()
customer["avg_ticket"]  = customer["monetary"] / customer["frequency"]
customer["max_ticket"]  = grp["LineTotal"].max()
customer["std_ticket"]  = grp["LineTotal"].std().fillna(0)
customer["n_products"]  = grp["StockCode"].nunique()

customer["last_purchase"]  = grp["InvoiceDate"].max()
customer["first_purchase"] = grp["InvoiceDate"].min()
customer["recency_days"]   = (reference_date - customer["last_purchase"]).dt.days
customer["tenure_days"]    = (customer["last_purchase"] - customer["first_purchase"]).dt.days

customer["country"] = grp["Country"].agg(lambda x: x.mode().iloc[0])

customer_features = customer.drop(columns=["last_purchase","first_purchase"])

customer_features.head()


,frequency,monetary,avg_ticket,max_ticket,std_ticket,n_products,recency_days,tenure_days,country
CustomerID,,,,,,,,,
12346,12,77556.46,6463.038333,77183.6,13234.943418,27,325,400,United Kingdom
12347,8,5633.32,704.165000,249.6,20.609093,126,1,402,Iceland
12348,5,2019.40,403.880000,240.0,44.205549,25,74,362,Finland
12349,4,4428.69,1107.172500,300.0,33.761403,138,18,570,Italy
12350,1,334.40,334.400000,40.0,7.275538,17,309,0,Norway


In [4]:
customer_features.to_csv("../data/processed/customer_features.csv")
customer_features.describe()


,frequency,monetary,avg_ticket,max_ticket,std_ticket,n_products,recency_days,tenure_days
count,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000,5878.000000
mean,6.289384,3018.616737,391.726538,186.945357,45.339900,81.989112,200.331916,273.022457
std,13.009406,14737.731040,1215.071054,2510.108739,1314.116430,116.484552,209.338707,258.807591
min,1.000000,2.950000,2.950000,2.950000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,348.762500,181.652500,34.680000,5.883088,19.000000,25.000000,0.000000
50%,3.000000,898.915000,285.070500,60.000000,9.749641,45.000000,95.000000,220.500000
75%,7.000000,2307.090000,420.570000,119.400000,18.775959,103.000000,379.000000,511.000000
max,398.000000,608821.650000,84236.250000,168469.600000,97265.131753,2550.000000,738.000000,738.000000


Ya contamos con un dataset a nivel cliente con variables de:

- Frecuencia de compra (`frequency`)
- Volumen monetario (`monetary`)
- Ticket promedio, máximo y volatilidad
- Diversificación de productos
- Recency y tenure
- País predominante

Este dataset se usará en el siguiente notebook para aplicar **clustering**.
